In [1]:
import pandas as pd 
import requests

In [2]:
df=pd.read_csv('/home/ubuntu/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset/reduced_metadata.csv')

In [3]:
df.head()

,audio_path,transcript,duration
0,844424930620027-1177-f_seg_0.wav,जेएनयू,0.000-1.524 sec
1,844424930620027-1177-f_seg_1.wav,में,1.564-1.845 sec
2,844424930620027-1177-f_seg_2.wav,हुई,1.885-2.126 sec
3,844424930620027-1177-f_seg_3.wav,हिंसा,2.166-2.687 sec
4,844424930620027-1177-f_seg_4.wav,को,2.727-2.888 sec


In [4]:
root_path='/home/ubuntu/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset'

In [20]:
! curl -X POST 'https://ai-tools.dev.bhasai.samagra.io/asr/bhashini_nisai/'  \
    -F "file=@/home/ubuntu/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset/844424930685385-130-f_seg_0.wav" \
    -F "text_read=[]" \
    -F "match_status=[]" \
    -F "fuzz_match=false" \
    -F "distance_cutoff=1" \
    -F "input_lang=hi"

{"match_status":[],"transcription":"\u0915"}


In [21]:
print("\u0915")

क


In [41]:
import requests
import os 
from tqdm import tqdm

def post_audio_file(audio_path):
    url = 'https://ai-tools.dev.bhasai.samagra.io/asr/bhashini_nisai/'
    files = {'file': open(audio_path, 'rb')}
    data = {
        'text_read': '[]',
        'match_status': '[]',
        'fuzz_match': 'false',
        'distance_cutoff': '1',
        'input_lang': 'hi'
    }

    response = requests.post(url, files=files, data=data)
    return response.json()

def generate_bhashani_transcript(df):
    transcript_list=[]
    for i in tqdm(range(len(df))):
        file_name=df['audio_path'][i]
        file_path=os.path.join(root_path,file_name)

        response=post_audio_file(file_path)

        transcript_list.append(response["transcription"])
    
    df['transcript_bhashini']=transcript_list

    df.to_csv(os.path.join(root_path,'bhashini_metadat.csv'))

generate_bhashani_transcript(df)
#post_audio_file(os.path.join(root_path,df['audio_path'][237]))

 17%|█▋        | 3934/23555 [22:13<1:50:51,  2.95it/s]  


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [45]:
file_path=os.listdir(root_path)
file_tasks=file_path[:10]

In [46]:
import asyncio
import aiohttp

def get_tasks(session):
    tasks=[]

    for file in file_tasks:
        url = 'https://ai-tools.dev.bhasai.samagra.io/asr/bhashini_nisai/'
        audio_path=os.path.join(root_path,file)
        files = {'file': open(audio_path, 'rb')}
        data = {
            'text_read': '[]',
            'match_status': '[]',
            'fuzz_match': 'false',
            'distance_cutoff': '1',
            'input_lang': 'hi'
        }

        tasks.append(asyncio.create_task(session.post(url, files=files, data=data)))

    return tasks

In [47]:
import json
async def get_bhashini_transcripts():

    results=[]
    async with aiohttp.ClientSession() as session:
        tasks=get_tasks(session)

        responses=await asyncio.gather(*tasks)

        for response in responses:
            results.append(await response.json()['transcription'])



In [48]:
asyncio.run(get_bhashini_transcripts())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [32]:
df['audio_path'][325]

'844424931273750-939-f_seg_6.wav'

In [27]:
df.head()

,audio_path,transcript,duration
0,844424930620027-1177-f_seg_0.wav,जेएनयू,0.000-1.524 sec
1,844424930620027-1177-f_seg_1.wav,में,1.564-1.845 sec
2,844424930620027-1177-f_seg_2.wav,हुई,1.885-2.126 sec
3,844424930620027-1177-f_seg_3.wav,हिंसा,2.166-2.687 sec
4,844424930620027-1177-f_seg_4.wav,को,2.727-2.888 sec


In [19]:
import IPython
import IPython.display
audio_path='/home/ubuntu/acoustic_stuff/hindi-acoustic-word-embedding/dataset/train_aligned_dataset/844424933559879-550-m_seg_13.wav'
IPython.display.Audio(audio_path,rate=16000)

In [14]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import os
# Apply the nested event loop fix
nest_asyncio.apply()

async def send_asr_request(session, file_path):
    url = 'https://ai-tools.dev.bhasai.samagra.io/asr/bhashini_nisai/'
    
    form = aiohttp.FormData()
    form.add_field('file', open(file_path, 'rb'), filename=file_path, content_type='audio/wav')
    form.add_field('text_read', '[]')
    form.add_field('match_status', '[]')
    form.add_field('fuzz_match', 'false')
    form.add_field('distance_cutoff', '1')
    form.add_field('input_lang', 'hi')
    
    async with session.post(url, data=form) as response:
        if response.headers['Content-Type'] == 'application/json':
            return await response.json()
        else:
            error_text = await response.text()
            return {'error': error_text}

async def process_files(df):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for index, row in df.iterrows():
            file_path = row['audio_path']
            file_path=os.path.join(root_path,file_path)
            tasks.append(asyncio.ensure_future(send_asr_request(session, file_path)))

        results = await asyncio.gather(*tasks)

        for i, result in enumerate(results):
            if 'error' in result:
                transcript_bhashini = f"Error: {result['error']}"
            else:
                transcript_bhashini = result.get('transcript', '')
            df.at[i, 'transcript_bhashini'] = transcript_bhashini


df['transcript_bhashini'] = ''

# Run the asynchronous processing
loop = asyncio.get_event_loop()
loop.run_until_complete(process_files(df))

print(df)





TimeoutError: 

In [3]:
df.head()

,Unnamed: 0,audio_path,transcript,duration
0,0,844424933492471-705-m_seg_0.wav,पिछली,0.000-0.926 sec
1,1,844424933492471-705-m_seg_2.wav,शताब्दियों,1.289-2.175 sec
2,2,844424933492471-705-m_seg_5.wav,विस्तृत,2.739-3.302 sec
3,3,844424933492471-705-m_seg_7.wav,बनकर,3.785-4.188 sec
4,6,844424930827743-329-f_seg_6.wav,विकास,2.255-2.698 sec


In [4]:
def get_batches(df, batch_size):
    num_batches = len(df) // batch_size + int(len(df) % batch_size != 0)
    for i in range(num_batches):
        yield df[i * batch_size:(i + 1) * batch_size]

In [5]:
batches=get_batches(df,500)

In [8]:
batch_1=next(iter(batches))

In [9]:
batch_1

,Unnamed: 0,audio_path,transcript,duration
500,574,844424931351202-1148-f_seg_6.wav,इशारे,4.588-5.031 sec
501,576,844424932544464-74-f_seg_0.wav,इससे,0.000-1.125 sec
502,577,844424932544464-74-f_seg_1.wav,प्रदेश,1.165-1.728 sec
503,578,844424932544464-74-f_seg_3.wav,परमपरागत,2.009-2.812 sec
504,579,844424932544464-74-f_seg_4.wav,कुटीर,2.893-3.294 sec
...,...,...,...,...
995,1162,844424932368379-1160-m_seg_2.wav,सुधारने,1.246-1.688 sec
996,1163,844424932368379-1160-m_seg_5.wav,विदेशी,2.050-2.452 sec
997,1164,844424932747215-645-f_seg_0.wav,उनका,0.000-0.565 sec
998,1165,844424932747215-645-f_seg_3.wav,प्रदर्शन,1.090-1.534 sec


In [21]:
for row in batch_1.iterrows():
    print(row[1][1])

844424931351202-1148-f_seg_6.wav
844424932544464-74-f_seg_0.wav
844424932544464-74-f_seg_1.wav
844424932544464-74-f_seg_3.wav
844424932544464-74-f_seg_4.wav
844424932544464-74-f_seg_9.wav
844424932544464-74-f_seg_12.wav
844424933458546-1179-m_seg_0.wav
844424933458546-1179-m_seg_1.wav
844424933458546-1179-m_seg_2.wav
844424933458546-1179-m_seg_4.wav
844424933458546-1179-m_seg_5.wav
844424933458546-1179-m_seg_6.wav
844424933458546-1179-m_seg_7.wav
844424933458546-1179-m_seg_8.wav
844424933458546-1179-m_seg_10.wav
844424933458546-1179-m_seg_12.wav
844424931055168-367-f_seg_1.wav
844424931055168-367-f_seg_4.wav
844424931055168-367-f_seg_5.wav
844424930729347-1148-f_seg_0.wav
844424930729347-1148-f_seg_1.wav
844424930729347-1148-f_seg_5.wav
844424930729347-1148-f_seg_9.wav
844424930583706-946-f_seg_0.wav
844424930583706-946-f_seg_4.wav
844424930583706-946-f_seg_7.wav
844424930583706-946-f_seg_14.wav
844424930865633-252-f_seg_0.wav
844424930865633-252-f_seg_2.wav
844424930865633-252-f_seg_5

/tmp/ipykernel_57153/1192779504.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[1][1])
